In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
import tensorflow as tf
from IPython.display import Image
warnings.filterwarnings('ignore')

#### 問題1

- 重み、バイアスの初期化、勾配計算、更新
- 層の定義(何層用意するか、パラメータ設定)
- 損失関数計算
- 正解率計算
- 学習率設定, Adagard実装
- 入力Xに対する予測
- 試行回数(epoch)のloop処理
- バッチで入力を分割して学習
- 活性関数定義
- フォワードプロパゲージョン、バックプロパゲージョンの実装
- 学習過程をdebug出力

In [2]:
# 問題2
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
tf.test.gpu_device_name() 

# データセットの読み込み
df = pd.read_csv("data/Iris.csv")

# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor") | (df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]

# NumPy 配列に変換
X = np.array(X)
y = np.array(y)
# ラベルを数値に変換
y[y == "Iris-versicolor"] = 0
y[y == "Iris-virginica"] = 1
y = y.astype(np.int64)[:, np.newaxis]

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

# ハイパーパラメータの設定
learning_rate = 0.001
batch_size = 10
num_epochs = 100

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

# eager_execution error回避
tf.compat.v1.disable_eager_execution()
# 計算グラフに渡す引数の形を決める
X = tf.compat.v1.placeholder("float", [None, n_input])
Y = tf.compat.v1.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    tf.random.set_seed(0)
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random.normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random.normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random.normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random.normal([n_hidden1])),
        'b2': tf.Variable(tf.random.normal([n_hidden2])),
        'b3': tf.Variable(tf.random.normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.compat.v1.global_variables_initializer()


# 計算グラフの実行
with tf.compat.v1.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
        total_loss /= n_samples
        val_loss, acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}".format(epoch, total_loss, val_loss, acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 7.0241, val_loss : 67.6859, acc : 0.375
Epoch 1, loss : 3.4241, val_loss : 23.4026, acc : 0.312
Epoch 2, loss : 1.9387, val_loss : 11.6681, acc : 0.375
Epoch 3, loss : 2.0917, val_loss : 13.1400, acc : 0.312
Epoch 4, loss : 1.7685, val_loss : 17.7284, acc : 0.312
Epoch 5, loss : 1.6097, val_loss : 12.9607, acc : 0.312
Epoch 6, loss : 1.4402, val_loss : 10.0593, acc : 0.312
Epoch 7, loss : 1.3704, val_loss : 9.4797, acc : 0.312
Epoch 8, loss : 1.2536, val_loss : 9.8518, acc : 0.312
Epoch 9, loss : 1.1476, val_loss : 8.5670, acc : 0.375
Epoch 10, loss : 1.0930, val_loss : 8.0430, acc : 0.375
Epoch 11, loss : 1.0412, val_loss : 7.8791, acc : 0.375
Epoch 12, loss : 0.9804, val_loss : 7.1233, acc : 0.375
Epoch 13, loss : 0.9326, val_loss : 6.7908, acc : 0.375
Epoch 14, loss : 0.8792, val_loss : 6.2492, acc : 0.375
Epoch 15, loss : 0.8304, val_loss : 5.7680, acc : 0.375
Epoch 16, loss : 0.7835, val_loss : 5.2886, acc : 0.438
Epoch 17, loss : 0.7384, val_loss : 4.8037, acc : 0

#### 問題2
scrach実装とtensorFlowとの対比

- 重み、バイアスの初期化: example_net内の定義
- 重み、バイアスの勾配計算、更新: train_op内の実装 `sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})`
- 層の定義(何層用意するか、パラメータ設定): example_netの定義
- 損失関数計算: `loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))`
- 正解率計算: `accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))`
- 学習率設定, Adagard実装: train_op内の実装
- 入力Xに対する予測: ??sklearnでいうpredictはなし
- 試行回数(epoch)のloop処理: session内のepochのloop
- バッチで入力を分割して学習: scrachと同じ `for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):`
- 活性関数定義: example_net内で各layerに定義
- フォワードプロパゲージョン、バックプロパゲージョンの実装: `sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})`内の処理
- 学習過程をdebug出力: epoch loopの最後

In [3]:
# データセットの読み込み
df = pd.read_csv("data/Iris.csv")

# データフレームから条件抽出
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]

# NumPy 配列に変換
X = np.array(X)
# y = np.array(y)
y = np.array(pd.get_dummies(df["Species"]))
# ラベルを数値に変換
# y[y == "Iris-setosa"] = 0
# y[y == "Iris-virginica"] = 1
# y[y == "Iris-versicolor"] = 2
# y = y.astype(np.int64)[:, np.newaxis]
# y = y[:, np.newaxis]

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [4]:
# ハイパーパラメータの設定
learning_rate = 0.001
batch_size = 10
num_epochs = 100

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 3

# eager_execution error回避
tf.compat.v1.disable_eager_execution()
# 計算グラフに渡す引数の形を決める
X = tf.compat.v1.placeholder("float", [None, n_input])
Y = tf.compat.v1.placeholder("float", [None, n_classes])

def Iris3_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    tf.random.set_seed(0)
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random.normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random.normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random.normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random.normal([n_hidden1])),
        'b2': tf.Variable(tf.random.normal([n_hidden2])),
        'b3': tf.Variable(tf.random.normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
logits = Iris3_net(X)

# 目的関数
loss_op = tf.reduce_mean(tf.compat.v1.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
correct_pred = tf.equal(tf.argmax(Y, axis=1), tf.argmax(logits, axis=1))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.compat.v1.global_variables_initializer()

In [5]:
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
# 計算グラフの実行
with tf.compat.v1.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
        total_loss /= n_samples
        val_loss, acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}".format(epoch, total_loss, val_loss, acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 11.3028, val_loss : 102.2221, acc : 0.292
Epoch 1, loss : 8.7998, val_loss : 81.4813, acc : 0.375
Epoch 2, loss : 6.8027, val_loss : 59.8930, acc : 0.542
Epoch 3, loss : 4.7791, val_loss : 39.9069, acc : 0.542
Epoch 4, loss : 3.0486, val_loss : 25.9002, acc : 0.208
Epoch 5, loss : 1.6053, val_loss : 10.0990, acc : 0.458
Epoch 6, loss : 0.4247, val_loss : 2.8406, acc : 0.708
Epoch 7, loss : 0.0561, val_loss : 1.1800, acc : 0.792
Epoch 8, loss : 0.0306, val_loss : 1.0144, acc : 0.875
Epoch 9, loss : 0.0183, val_loss : 0.7311, acc : 0.917
Epoch 10, loss : 0.0175, val_loss : 0.8112, acc : 0.917
Epoch 11, loss : 0.0155, val_loss : 0.7299, acc : 0.917
Epoch 12, loss : 0.0162, val_loss : 0.8065, acc : 0.917
Epoch 13, loss : 0.0145, val_loss : 0.7146, acc : 0.917
Epoch 14, loss : 0.0157, val_loss : 0.8388, acc : 0.917
Epoch 15, loss : 0.0131, val_loss : 0.6572, acc : 0.917
Epoch 16, loss : 0.0157, val_loss : 0.8889, acc : 0.917
Epoch 17, loss : 0.0118, val_loss : 0.6051, acc : 

In [6]:
# 問題4
from sklearn.preprocessing import StandardScaler
# データセットの読み込み
df = pd.read_csv("data/housing/train.csv")

# データフレームから条件抽出
y = df["SalePrice"]
X = df.loc[:, ["GrLivArea", "YearBuilt"]]

# NumPy 配列に変換
X = np.array(X)
y = np.array(y)
y = y[:, np.newaxis]

# 標準化
standardScaler_y = StandardScaler()
standardScaler_X = StandardScaler()
y = standardScaler_y.fit_transform(y)
X = standardScaler_X.fit_transform(X)

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [7]:
# ハイパーパラメータの設定
learning_rate = 0.001
batch_size = 10
num_epochs = 100

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

# eager_execution error回避
tf.compat.v1.disable_eager_execution()
# 計算グラフに渡す引数の形を決める
X = tf.compat.v1.placeholder("float", [None, n_input])
Y = tf.compat.v1.placeholder("float", [None, n_classes])

def housingPrice_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    tf.random.set_seed(0)
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random.normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random.normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random.normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random.normal([n_hidden1])),
        'b2': tf.Variable(tf.random.normal([n_hidden2])),
        'b3': tf.Variable(tf.random.normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
logits = housingPrice_net(X)

# 目的関数
loss_op = tf.compat.v1.losses.mean_squared_error(labels=Y, predictions=logits)
# 最適化手法
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# variableの初期化
init = tf.compat.v1.global_variables_initializer()

In [8]:
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
pred_price = None
# 計算グラフの実行
with tf.compat.v1.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss = sess.run(loss_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
        total_loss /= n_samples
        val_loss = sess.run(loss_op, feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}".format(epoch, total_loss, val_loss))
        pred_price = logits
    test_loss = sess.run(loss_op, feed_dict={X: X_test, Y: y_test})
    print("loss_op : {:.3f}".format(test_loss))

Epoch 0, loss : 24.5132, val_loss : 59.0078
Epoch 1, loss : 4.0381, val_loss : 15.6652
Epoch 2, loss : 1.5786, val_loss : 7.6257
Epoch 3, loss : 0.8941, val_loss : 5.2186
Epoch 4, loss : 0.6093, val_loss : 4.2106
Epoch 5, loss : 0.4659, val_loss : 3.6830
Epoch 6, loss : 0.3809, val_loss : 3.3319
Epoch 7, loss : 0.3218, val_loss : 3.0504
Epoch 8, loss : 0.2782, val_loss : 2.8167
Epoch 9, loss : 0.2433, val_loss : 2.5851
Epoch 10, loss : 0.2154, val_loss : 2.3353
Epoch 11, loss : 0.1892, val_loss : 2.0991
Epoch 12, loss : 0.1694, val_loss : 1.9041
Epoch 13, loss : 0.1545, val_loss : 1.7625
Epoch 14, loss : 0.1426, val_loss : 1.6403
Epoch 15, loss : 0.1336, val_loss : 1.5269
Epoch 16, loss : 0.1250, val_loss : 1.4517
Epoch 17, loss : 0.1185, val_loss : 1.3770
Epoch 18, loss : 0.1127, val_loss : 1.3043
Epoch 19, loss : 0.1075, val_loss : 1.2504
Epoch 20, loss : 0.1030, val_loss : 1.2006
Epoch 21, loss : 0.0988, val_loss : 1.1451
Epoch 22, loss : 0.0952, val_loss : 1.0918
Epoch 23, loss : 0

In [9]:
# TODO 実際の予測値が見たいがtf.Tensorをnumpyで出力する方法がわからない。 .numpy()はエラー
pred_price

<tf.Tensor 'add_8:0' shape=(None, 1) dtype=float32>

In [10]:
# 問題5
# データセットの読み込み
from keras.datasets import mnist
from sklearn.preprocessing import OneHotEncoder
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)
# データフレームから条件抽出

# 0 1スケールに変更
X_train = X_train / 255

# 1次元に変換
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1] ** 2))
print("X_train reshaped: ", X_train.shape)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1] ** 2))
print("X_test reshaped: ", X_test.shape)
# y次元変換

ohe = OneHotEncoder()

y_train = y_train[:, np.newaxis]
y_test = y_test[:, np.newaxis]
y_train = ohe.fit_transform(y_train).toarray()
y_test = ohe.transform(y_test).toarray()
print("y_train: ", y_train.shape)
print("y_test: ", y_test.shape)

# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

X_train:  (60000, 28, 28)
X_test:  (10000, 28, 28)
X_train reshaped:  (60000, 784)
X_test reshaped:  (10000, 784)
y_train:  (60000, 10)
y_test:  (10000, 10)


In [12]:
y_train

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [13]:
# ハイパーパラメータの設定
learning_rate = 0.001
batch_size = 10
num_epochs = 100

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 10

# eager_execution error回避
tf.compat.v1.disable_eager_execution()
# 計算グラフに渡す引数の形を決める
X = tf.compat.v1.placeholder("float", [None, n_input])
Y = tf.compat.v1.placeholder("float", [None, n_classes])

def mnist_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    tf.random.set_seed(0)
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random.normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random.normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random.normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random.normal([n_hidden1])),
        'b2': tf.Variable(tf.random.normal([n_hidden2])),
        'b3': tf.Variable(tf.random.normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
logits = mnist_net(X)

# 目的関数
loss_op = tf.reduce_mean(tf.compat.v1.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
correct_pred = tf.equal(tf.argmax(Y, axis=1), tf.argmax(logits, axis=1))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.compat.v1.global_variables_initializer()

In [14]:
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
# 計算グラフの実行
with tf.compat.v1.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
        total_loss /= n_samples
        val_loss, acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}".format(epoch, total_loss, val_loss, acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 2.7981, val_loss : 8.1544, acc : 0.799
Epoch 1, loss : 0.4917, val_loss : 3.6453, acc : 0.830
Epoch 2, loss : 0.2260, val_loss : 1.9663, acc : 0.840
Epoch 3, loss : 0.1227, val_loss : 1.2979, acc : 0.848
Epoch 4, loss : 0.0785, val_loss : 1.0034, acc : 0.863
Epoch 5, loss : 0.0585, val_loss : 0.8404, acc : 0.873
Epoch 6, loss : 0.0466, val_loss : 0.7679, acc : 0.884
Epoch 7, loss : 0.0392, val_loss : 0.7292, acc : 0.889
Epoch 8, loss : 0.0337, val_loss : 0.6963, acc : 0.894
Epoch 9, loss : 0.0299, val_loss : 0.6603, acc : 0.902
Epoch 10, loss : 0.0269, val_loss : 0.6562, acc : 0.902
Epoch 11, loss : 0.0245, val_loss : 0.6385, acc : 0.907
Epoch 12, loss : 0.0229, val_loss : 0.5725, acc : 0.910
Epoch 13, loss : 0.0210, val_loss : 0.6171, acc : 0.911
Epoch 14, loss : 0.0196, val_loss : 0.5930, acc : 0.918
Epoch 15, loss : 0.0184, val_loss : 0.5959, acc : 0.919
Epoch 16, loss : 0.0173, val_loss : 0.6007, acc : 0.917
Epoch 17, loss : 0.0161, val_loss : 0.5915, acc : 0.918
Ep